In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')
import sys
import os
prefix = '/content/gdrive/My Drive/'
# modify "customized_path_to_your_homework" here to where you uploaded your homework
customized_path_to_your_homework = 'IDLSProject-main'
sys_path = os.path.join(prefix, customized_path_to_your_homework)
sys.path.append(sys_path)

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [2]:
%cd '/content/gdrive/My Drive/IDLSProject-main-2'

/content/gdrive/My Drive/IDLSProject-main-2


In [3]:
import pandas as pd
import os
import pickle

data_dir = './data/pytorch'
with open(os.path.join(data_dir, 'word_dict_amazon.pkl'), "rb") as f:
    word_dict = pickle.load(f)



In [4]:
import pandas as pd
import os
train = pd.read_csv(os.path.join(data_dir, 'train_amazon.csv'), header=None, names=None)
test_sample = pd.read_csv(os.path.join(data_dir, 'test_amazon.csv'), header=None, names=None)
print(train.shape, test_sample.shape)

(83000, 502) (21975, 502)


In [5]:
from sklearn.model_selection import train_test_split
test, val = train_test_split(test_sample, test_size=0.5)
train.shape, test.shape, val.shape

((83000, 502), (10987, 502), (10988, 502))

In [6]:
import torch
import torch.utils.data

#drop rows
test.drop(test.tail(37).index,inplace = True)
val.drop(val.tail(38).index,inplace = True)
test.shape, val.shape


# Turn the input pandas dataframe into tensors
train_y = torch.from_numpy(train[[0]].values).float()
train_X = torch.from_numpy(train.drop([0, 1], axis=1).values).long()

# Build the dataset
train_ds = torch.utils.data.TensorDataset(train_X, train_y)
# Build the dataloader
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=50)

######val data
# Turn the input pandas dataframe into tensors
val_y = torch.from_numpy(val[[0]].values).float()
val_X = torch.from_numpy(val.drop([0, 1], axis=1).values).long()

# Build the dataset
val_ds = torch.utils.data.TensorDataset(val_X, val_y)
# Build the dataloader
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=50)


#### Test data
# Turn the input pandas dataframe into tensors
test_y = torch.from_numpy(test[[0]].values).float()
test_X = torch.from_numpy(test.drop([0, 1], axis=1).values).long()

# Build the dataset
test_ds = torch.utils.data.TensorDataset(test_X, test_y)
# Build the dataloader
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=50)
print(test_y.shape)

torch.Size([10950, 1])


In [7]:
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

class QRNNLayer(nn.Module):
    def __init__(self,batch_size,input_size,n_filters,kernel_size,embed_size,device,dropout):
        super(QRNNLayer,self).__init__()
        self.batch_size = batch_size
        self.input_size = input_size
        self.n_filters = n_filters
        self.kernel_size = kernel_size
        self.embed_size = embed_size
        self.dropout = torch.nn.Dropout(dropout)
        self.device = device
        self.conv1 = torch.nn.Conv1d(self.input_size,self.n_filters,self.kernel_size)
        self.conv2 = torch.nn.Conv1d(self.input_size,self.n_filters,self.kernel_size)
        self.conv3 = torch.nn.Conv1d(self.input_size,self.n_filters,self.kernel_size)
    
    def forward(self,masked_input, h, c):
        Z,F,O = self.masked_conv(masked_input)
        h, c = self.pool(c,Z,F,O)
        masked_input = h
        return masked_input,h,c
    
    def masked_conv(self,x):
        pad = torch.zeros([self.batch_size,1,self.input_size],device=self.device)
        x = torch.cat([pad,x],1).permute(0,2,1)
        Z = torch.tanh((self.conv1(x)))
        F = torch.sigmoid((self.conv2(x)))
        O = torch.sigmoid((self.conv3(x)))
        one_mask = torch.ones_like(F,device=self.device) - F
        F = 1 - self.dropout(one_mask)
        return Z.permute(0,2,1), F.permute(0,2,1), O.permute(0,2,1)
    
    def pool(self, prev_c,Z,F,O):
        c = torch.mul(F,prev_c) + torch.mul(1-F,Z)
        h = torch.mul(O,c)
        return h,c

class QRNN(nn.Module):
    def __init__(self,trial,vocab_size,embed_size,n_filters,kernel_size,batch_size,seq_len,layers,device,dropout):
        super(QRNN,self).__init__()
        dropout = trial.suggest_uniform("dropout",0.1, 0.6)
        self.num_layer = trial.suggest_int("num_layer",1,4)
        self.embed_size = trial.suggest_int("embed_size",16,128)
        self.n_filters = trial.suggest_int("n_filters",50,200)
        # self.kernel_size = trial.suggest_int("kernel_size",2,5)
        # self.embed_size = embed_size
        # self.n_filters = n_filters
        self.kernel_size = kernel_size
        self.batch_size = batch_size
        self.seq_len = seq_len
        # self.num_layer = layers
        self.device = device
        self.embedding = torch.nn.Embedding(vocab_size, self.embed_size)
        self.dense = torch.nn.Linear(self.seq_len*self.n_filters,1)
        self.QRNN_layers = torch.nn.ModuleList([QRNNLayer(self.batch_size,self.embed_size if l==0 else self.n_filters,
                                                         self.n_filters,self.kernel_size,self.embed_size,self.device,
                                                         dropout,) for l in range(self.num_layer)])
        
        
    def forward(self, x, target):
        x = self.embedding(x)
        h = torch.zeros([self.batch_size,self.seq_len,self.n_filters],device=self.device)
        c = torch.zeros_like(h,device=self.device)
        
        masked_input = x
        for l,layer in enumerate(self.QRNN_layers):
            masked_input,h,c = layer(masked_input,h,c)
        dense_input = h.reshape([self.batch_size,-1])
        logits = self.dense(dense_input)
        prediction = torch.sigmoid(logits)
        target = target.view([-1,1])
        correct_pred = torch.eq(torch.round(prediction).type(target.type()),target)
        accuracy = torch.sum(correct_pred)
        return prediction, accuracy

In [8]:
data_dir = './qrnn-amazon-hyperband-trails/' # The folder we will use for storing data
if not os.path.exists(data_dir): # Make sure that the folder exists
    os.makedirs(data_dir)

filename = ""
def write_to_csv(trail_num, epochs, train_loss, train_acc, val_loss, val_acc, time_train):
    global filename
    filename = "./qrnn-amazon-hyperband-trails/"+str(trail_num)+".csv"
    epoch = [i for i in range(epochs)]
    df_metrics = pd.DataFrame(list(zip(epoch, train_loss, train_acc, val_loss, val_acc, time_train)), columns =['Epoch', 'train_loss', 'train_acc', 'val_loss', 'val_acc', 'train_time'])
    df_metrics.to_csv(filename, index=False)    
    
def append_to_csv(epochs, accuracy):
    acc = [accuracy for i in range(epochs)]
    df_csv = pd.read_csv(filename)
    df_csv['Test_Accuracy']  = acc
    df_csv.to_csv(filename, index=False)

In [9]:
!pip install optuna

In [10]:
import time
epochs = 10
n_filters = 64
kernel_size = 2
layers= 2
learning_rate = 0.001
print(len(word_dict))
vocab_size = 5000
print(vocab_size)
embed_dims = 32
seq_len = 500
dropout = 0.3
batch_size=50

9998
5000


In [11]:
import time
import optuna
from optuna.trial import TrialState
import torch.optim as optim
import torch.optim as optim
import numpy as np

# Define an objective function to be minimized.
def objective(trial):
  optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
  #epochs = trial.suggest_int("epochs",5,15)
  lr = trial.suggest_loguniform("lr", 1e-5, 1e-0)
  #momentum = trial.suggest_uniform("momentum", 0.0, 1.0)
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model = QRNN(trial,vocab_size, embed_dims, n_filters, kernel_size, batch_size, seq_len, layers, device, dropout).to(device)
  trial.set_user_attr(key="best_model", value=model)
  optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
  loss_fn = torch.nn.BCELoss()
  counter = 0
  QRNN_acc = []
  QRNN_valacc = []
  train_loss_epoch = []
  train_acc_epoch = []
  val_loss_epoch = []
  val_acc_epoch = []
  time_epoch = []
  final_val_acc = 0
  model.train()
  for e in range(epochs):
    start_time = time.time()
    train_loss = []
    train_acc = []
    for inputs, labels in train_dl:
        inputs, labels = inputs.cuda(), labels.cuda()
        model.zero_grad()
        logits, accuracy = model(inputs,labels)
        loss = criterion(logits,labels.float())
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 5)
        optimizer.step()
        train_loss.append(loss.item())
        train_acc.append(accuracy.item()*100/batch_size)
        if counter%100==0:
            print("Epoch: {}/{}".format(e,epochs),
                         "\tIteration: {}".format(counter),
                         "\tTrain Loss: {:.3f}".format(loss.item()),
                         "\tTrain Accuracy: {:.2f}".format(accuracy.item()*100/batch_size))
            QRNN_acc.append(accuracy.item()*100/batch_size)
        counter += 1
    train_loss_epoch.append(np.round(np.mean(train_loss), 3))
    train_acc_epoch.append(np.round(np.mean(train_acc), 3))
    print("\tTrain Loss: {:.3f}".format(np.mean(train_loss)), "\tTrain Acc: {:.3f}".format(np.mean(train_acc)))
        
    with torch.no_grad():
        model.eval()
        val_acc = []
        val_loss = []
        for inputs, labels in val_dl:
            inputs_val, labels_val = inputs.cuda(), labels.cuda()
            logits_val,accuracy_val = model(inputs_val,labels_val)
            loss_val = criterion(logits_val,labels_val.float())
            val_acc.append(accuracy_val.item()*100/batch_size)
            val_loss.append(loss_val.item())
        print("Val Loss: {:.3f}".format(np.mean(val_loss)), "\tVal Acc: {:.3f}".format(np.mean(val_acc)))
        val_loss_epoch.append(np.round(np.mean(val_loss), 3))
        val_acc_epoch.append(np.round(np.mean(val_acc), 3))
        QRNN_valacc.append(np.mean(val_acc))
        model.train()
        final_val_acc = np.mean(val_acc)
    end_time = time.time()-start_time
    print("Time to train epoch: {0} s".format(end_time))
    time_epoch.append(np.round(end_time, 3))
  write_to_csv(epochs, train_loss_epoch, train_acc_epoch, val_loss_epoch, val_acc_epoch, time_epoch)
  return final_val_acc


In [12]:
def callback(study, trial):
    if study.best_trial.number == trial.number:
        study.set_user_attr(key="best_model", value=trial.user_attrs["best_model"])

In [13]:
import optuna
from optuna.trial import TrialState
study = optuna.create_study(direction="maximize",pruner=optuna.pruners.HyperbandPruner(
        min_resource=1, max_resource=epochs, reduction_factor=3
    ),)
study.optimize(objective, n_trials=5,callbacks=[callback])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])
print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of complete trials: ", len(complete_trials))
print("Best trial:")
trial = study.best_trial
best_model=study.user_attrs["best_model"]
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
  print("    {}: {}".format(key, value))

[I 2022-05-14 18:38:09,841] A new study created in memory with name: no-name-0441cda1-e091-4fe8-9bc8-333d01a29d1a
[W 2022-05-14 18:38:17,730] Trial 0 failed because of the following error: RuntimeError('CUDA error: device-side assert triggered\nCUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.\nFor debugging consider passing CUDA_LAUNCH_BLOCKING=1.')
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/optuna/study/_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-11-bf2c44b7c861>", line 36, in objective
    logits, accuracy = model(inputs,labels)
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py", line 1110, in _call_impl
    return forward_call(*input, **kwargs)
  File "<ipython-input-7-9c45b7a661ad>", line 70, in forward
    masked_input,h,c = layer(masked_input,h,c)
  File "/usr/local/lib/python3.7/dist-packages/torc

RuntimeError: ignored

In [ ]:
data_dir = './qrnn-amazon-hyperband/' # The folder we will use for storing data
if not os.path.exists(data_dir): # Make sure that the folder exists
    os.makedirs(data_dir)
    
torch.save(best_model.state_dict(),"./qrnn-amazon-hyperband/qrnn-imdb-hyperband.pth")

In [ ]:
def test(model, test_dl, epochs):
    with torch.no_grad():
    model.eval()
    test_acc = []
    test_loss = []
    for inputs, labels in test_dl:
        input_test, labels_test = inputs.cuda(), labels.cuda()
        logits_test, accuracy_test = model(input_test, labels_test)
        loss_test = criterion(logits_test, labels_test.float())
        test_acc.append(accuracy_test.item()*100/batch_size)
        test_loss.append(loss_test.item())
    print("Test Loss: {:.3f}".format(np.mean(test_loss)), "\tTest Acc: {:.3f}".format(np.mean(test_acc)))

In [ ]:
test(best_model, test_dl, epochs)